Building a transformer model for any language starts from loading the data and building a tokenizer (unless a pre-trained model exists, e.g. for English). In this set of Jupyter Notebooks I follow closely an awesome report "How to Train a BERT Model From Scratch. Meet BERT’s Italian cousin, FiliBERTo" by James Bridge: https://towardsdatascience.com/how-to-train-a-bert-model-from-scratch-72cfce554fc6

### Get the data
For training a transformer model we can use any "raw" text. A nice collection of such texts in 100+ languages is OSCAR dataset which is also free:

In [1]:
import datasets

In [2]:
datasetUkr = datasets.load_dataset('oscar', 'unshuffled_deduplicated_uk')

Downloading:   0%|          | 0.00/1.32k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/516M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/520M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/520M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/522M [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/526M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/525M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/198M [00:00<?, ?B/s]

0 examples [00:00, ? examples/s]

Dataset oscar downloaded and prepared to C:\Users\Asus\.cache\huggingface\datasets\oscar\unshuffled_deduplicated_uk\1.0.0\84838bd49d2295f62008383b05620571535451d84545037bb94d6f3501651df2. Subsequent calls will reuse this data.


In [3]:
datasetUkr

DatasetDict({
    train: Dataset({
        features: ['id', 'text'],
        num_rows: 7782375
    })
})

In [5]:
datasetUkr['train'][10]

{'id': 10,
 'text': 'Суд позбавив А. і Волкова офіцерських звань і вийшов з представленням до ПВР СРСР про позбавлення їх державних нагород, також постановив знищити рукописи А. і Волкова «как предметы, не представляющие никакой ценности».\n22.12.1982 р. А. етапом вивезений у Мордовію. У табір ЖХ-385/3-4, що в сел. Барашево Теньгушовського р-ну, прибув 30.12. Тут були політв`язні В. СТРІЛЬЦІВ, Ю. БАДЗЬО, Ю. МЕЛЬНИК, Д. МАЗУР, пізніше Володимир Делидівка, Г. КУЦЕНКО, Євген Анцупов, Г. АЛТУНЯН.\nОскільки вирок А. був надто інформативний (28 с.), його на руки не дали. Зона ж вимагала: «Покажи вирок». Тоді А. попросив спецчастину надати йому вирок для написання скарги. Його зачинили під замок, але він передав вирок через кватирку в зону. Його прочитали і повернули.\nПолітв΄язні-«антирадянщики» (чоловік 30–40) становили кістяк табору. Вивчали юриспруденцію, писали скарги та заяви, відзначати голодівками День політв\x92язня – 30 жовтня та 10 грудня – День прав людини, сиділи в карцерах. По н

In [6]:
from tqdm.auto import tqdm

# split each text file into chunks of 5K samples each 
# save them into a new oscar_la directory

text_data = []
file_count = 0

for sample in tqdm(datasetUkr['train']):
    # remove newline characters from each sample as we need to use exclusively as seperators
    sample = sample['text'].replace('\n', '')
    text_data.append(sample)
    if len(text_data) == 5_000:
        # once we hit the 5K mark, save to file
        with open(f'./data/text/oscar_uk/text_{file_count}.txt', 'w', encoding='utf-8') as fp:
            fp.write('\n'.join(text_data))
        text_data = []
        file_count += 1
# after saving in 5K chunks, we will have ~3808 leftover samples, we save those now too
with open(f'./data/text/oscar_uk/text_{file_count}.txt', 'w', encoding='utf-8') as fp:
    fp.write('\n'.join(text_data))

  0%|          | 0/7782375 [00:00<?, ?it/s]

In [7]:
from pathlib import Path
paths = [str(x) for x in Path('./data/text/oscar_uk').glob('**/*.txt')]
paths

['data\\text\\oscar_uk\\text_0.txt',
 'data\\text\\oscar_uk\\text_1.txt',
 'data\\text\\oscar_uk\\text_10.txt',
 'data\\text\\oscar_uk\\text_100.txt',
 'data\\text\\oscar_uk\\text_1000.txt',
 'data\\text\\oscar_uk\\text_1001.txt',
 'data\\text\\oscar_uk\\text_1002.txt',
 'data\\text\\oscar_uk\\text_1003.txt',
 'data\\text\\oscar_uk\\text_1004.txt',
 'data\\text\\oscar_uk\\text_1005.txt',
 'data\\text\\oscar_uk\\text_1006.txt',
 'data\\text\\oscar_uk\\text_1007.txt',
 'data\\text\\oscar_uk\\text_1008.txt',
 'data\\text\\oscar_uk\\text_1009.txt',
 'data\\text\\oscar_uk\\text_101.txt',
 'data\\text\\oscar_uk\\text_1010.txt',
 'data\\text\\oscar_uk\\text_1011.txt',
 'data\\text\\oscar_uk\\text_1012.txt',
 'data\\text\\oscar_uk\\text_1013.txt',
 'data\\text\\oscar_uk\\text_1014.txt',
 'data\\text\\oscar_uk\\text_1015.txt',
 'data\\text\\oscar_uk\\text_1016.txt',
 'data\\text\\oscar_uk\\text_1017.txt',
 'data\\text\\oscar_uk\\text_1018.txt',
 'data\\text\\oscar_uk\\text_1019.txt',
 'data\\te

In [8]:
len(paths)

1557

### Train the Tokenizer

In [9]:
from tokenizers import ByteLevelBPETokenizer
# initialize
tokenizer = ByteLevelBPETokenizer()

In [10]:
tokenizer.train(files=paths, vocab_size=30_522, min_frequency=2,
                special_tokens=['<s>', '<pad>', '</s>', '<unk>', '<mask>'])

In [11]:
import os

os.mkdir('panas')
tokenizer.save_model('panas')

['panas\\vocab.json', 'panas\\merges.txt']